In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

import pandas as pd
import joblib
import json
import numpy as np

sys.path.append('..')

In [2]:
json_file = "/mnt/data/stephana93dm/storage/projects/alternative_clustering/alternative_clustering/datasets/NR-Objects/scenes/CLEVR_train_000000.json"


data_dir = "/mnt/data/stephana93dm/storage/projects/alternative_clustering/alternative_clustering/datasets/"
dataset_name = "Cards"
load_dir = os.path.join(data_dir, dataset_name)
save_dir = os.path.join(data_dir, "processed", dataset_name)

img_dir = os.path.join(load_dir, "instance")

In [3]:
df = []
for fold in os.listdir(img_dir):
    fold_dir = os.path.join(img_dir, fold)
    if not os.path.isdir(fold_dir):
        continue
    assert len(os.listdir(fold_dir)) == 1
    for img in os.listdir(fold_dir):
        img_path = os.path.join(fold_dir, img)

        number = img.split(" ")[0]
        suit = img.split(" ")[2].split("_")[0]
        # df.append({"path": img_path, "fold": fold})
        df.append({"image_filename": img, "image_path": img_path, "number": number, "suit": suit})

df = pd.DataFrame(df)
df

,image_filename,image_path,number,suit
0,seven of spades_125.jpg,/mnt/data/stephana93dm/storage/projects/altern...,seven,spades
1,two of diamonds_121.jpg,/mnt/data/stephana93dm/storage/projects/altern...,two,diamonds
2,six of spades_80.jpg,/mnt/data/stephana93dm/storage/projects/altern...,six,spades
3,queen of spades_112.jpg,/mnt/data/stephana93dm/storage/projects/altern...,queen,spades
4,two of clubs_122.jpg,/mnt/data/stephana93dm/storage/projects/altern...,two,clubs
...,...,...,...,...
8024,seven of hearts_70.jpg,/mnt/data/stephana93dm/storage/projects/altern...,seven,hearts
8025,four of clubs_164.jpg,/mnt/data/stephana93dm/storage/projects/altern...,four,clubs
8026,king of hearts_103.jpg,/mnt/data/stephana93dm/storage/projects/altern...,king,hearts
8027,jack of diamonds_51.jpg,/mnt/data/stephana93dm/storage/projects/altern...,jack,diamonds


In [4]:
from turtle import color
from sklearn.preprocessing import LabelEncoder

number_le = LabelEncoder()
suit_le = LabelEncoder()

df["number_clusters"] = number_le.fit_transform(df["number"])
df["suit_clusters"] = suit_le.fit_transform(df["suit"])

cluster_to_name = {
    "number": {i: name for i, name in enumerate(number_le.classes_)},
    "suit": {i: name for i, name in enumerate(suit_le.classes_)},
}

In [5]:
import shutil


save_folder = save_dir
image_save_folder = os.path.join(save_folder, "images")
clustering_save_folder = os.path.join(save_folder, "clustering")
os.makedirs(image_save_folder, exist_ok=True)
os.makedirs(clustering_save_folder, exist_ok=True)

with open(os.path.join(save_folder, "cluster_to_name.json"), "w") as f:
    json.dump(cluster_to_name, f)

for cluster_name in ["number", "suit"]:
    dictionary = {}
    for idx, row in df.iterrows():
        name = row["image_filename"]
        dictionary[name] = row[f"{cluster_name}_clusters"]

        src = row["image_path"]
        dst = os.path.join(image_save_folder, name)
        shutil.copyfile(src, dst)
    
    save_file = os.path.join(clustering_save_folder, f"clusters_{cluster_name}.json")
    with open(save_file, "w") as f:
        json.dump(dictionary, f)